In [1]:
import time #шаблон времени
startall_time = time.time()## точка отсчета времени

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import *
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import sys
from scipy import sqrt, pi, arctan2, cos, sin
from scipy.ndimage import uniform_filter

In [2]:
def getNeighborhood(img, idx, idy, default=0):
    try:
        return img[idx,idy]
    except IndexError:
        return default

def thresholded(center, pixels):
    ret = []
    for a in pixels:
        if a >= center:
            ret.append(1)
        else:
            ret.append(0)
    return ret


def lbp_c(img):
    height, width = img.shape
    ret = np.empty([height, width], np.uint8)
    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            center        = img[x,y]
            top_left      = getNeighborhood(img, x-1, y-1)
            top_middle    = getNeighborhood(img, x, y-1)
            top_right     = getNeighborhood(img, x+1, y-1)
            right         = getNeighborhood(img, x+1, y )
            left          = getNeighborhood(img, x-1, y )
            bottom_left   = getNeighborhood(img, x-1, y+1)
            bottom_right  = getNeighborhood(img, x+1, y+1)
            bottom_middle = getNeighborhood(img, x,   y+1)

            values = thresholded(center, [top_left, top_middle, top_right, right, bottom_right,
                                          bottom_middle, bottom_left, left])

            weights = [1, 2, 4, 8, 16, 32, 64, 128]
            res = 0
            for a in range(0, len(values)):
                res += weights[a] * values[a]
            ret.itemset((x,y), res)
    return ret

## Load and preprocessing image ##

In [3]:
def LaP_images(path):
    images = []
    filenames = os.listdir(path)
    for filename in filenames:    
        image = cv2.imread(os.path.join(path, filename),0)
        image = cv2.resize(image, dsize=(150,150))
        images.append(image)
    return np.array(images)

In [4]:
start_LaP_time = time.time()## точка отсчета времени
foil_train_dir = os.getcwd() + "\\Desert"
corduroy_train_dir = os.getcwd() + "\\DenseResidential"
cork_train_dir = os.getcwd() + "\\Forest"
catton_train_dir = os.getcwd() + "\\Meadow"
linen_train_dir = os.getcwd() + "\\River"

In [5]:
foil_train_list = LaP_images(foil_train_dir)
corduroy_train_list = LaP_images(corduroy_train_dir)
cork_train_list = LaP_images(cork_train_dir)
catton_train_list = LaP_images(catton_train_dir)
linen_train_list = LaP_images(linen_train_dir)

In [6]:
print(len(foil_train_list[1]))
print(len(foil_train_list))
print(len(corduroy_train_list))
print(len(cork_train_list))
print(len(catton_train_list))
print(len(linen_train_list))

150
217
217
217
217
217


### All image in one array ###

In [7]:
start_EX_time = time.time()## точка отсчета времени
all_images = np.vstack((foil_train_list,
                        corduroy_train_list,
                        cork_train_list,
                        catton_train_list,
                        linen_train_list
                        ))
print(all_images.shape,"Shape before extract")

(1085, 150, 150) Shape before extract


In [8]:
labels = []
for i in range(len(foil_train_list)):
    labels.append(0)
for i in range(len(corduroy_train_list)):
    labels.append(1)
for i in range(len(cork_train_list)):
    labels.append(2)
for i in range(len(catton_train_list)):
    labels.append(3)
for i in range(len(linen_train_list)):
    labels.append(4)
    
labels = np.array(labels)
print("--- %s seconds Load and preprocessing ---" % (time.time() - start_LaP_time)) ## вывод время работы программы

--- 3.2479512691497803 seconds Load and preprocessing ---


## Extract LBP features ##

In [9]:
def extract_lbp(images):
    lbps = []
    for image in images:
        lbp = lbp_c(image)
        lbps.append(lbp)
    
    return np.array(lbps)

In [10]:

start_EX_time = time.time()## точка отсчета времени
X_all_lbp = extract_lbp(all_images)
print("--- %s seconds Extract LBP ---" % (time.time() - start_EX_time)) ## вывод время работы программы

--- 78.31314444541931 seconds Extract LBP ---


In [11]:
X_all_lbp.shape

(1085, 150, 150)

 ### Making general histogram ###

In [12]:
def create_histograms(images, sub_images_num, bins_per_sub_images):
    all_histograms = []
    for image in images:
        grid = np.arange(0, image.shape[1]+1, image.shape[1]//sub_images_num) #Возвращает равномерно распределенные значения в пределах заданного интервала.(0,151,50) sub_images_num-на сколько областей^2

        sub_image_histograms = []

        for i in range(1, len(grid)):
            for j in range(1, len(grid)):
                sub_image = image[grid[i-1]:grid[i], grid[j-1]:grid[j]]

                sub_image_histogram = np.histogram(sub_image, bins=bins_per_sub_images)[0] # разбивает диапозон на 64 части(шаг = 4) и смотрит кол-во вхождекний
                sub_image_histograms.append(sub_image_histogram)

        histogram = np.array(sub_image_histograms).flatten()
        all_histograms.append(histogram)
        
    return np.array(all_histograms)

>1.Разбиваем изображение 150:150 на 9 областей(50:50)(строим сетку) <br>
2.Проходимся по каждой области и строим там гистограмму распредления(кол-во вхождекний) получившихся "переходов света лбп". Сами задаем интервалы в гистограмме(bins) <br>
3.Гистограмму из каждой области добавляем в лист и склеиваем. <br>
4.Получаем общую гистограмму изображения.

In [13]:
start_HIST_time = time.time()## точка отсчета времени

X_train_hist = create_histograms(X_all_lbp, sub_images_num=3, bins_per_sub_images=84)


print('X_train_hist\t', X_train_hist.shape)

print("--- %s seconds Histofram ---" % (time.time() - start_HIST_time)) ## вывод время работы программы

X_train_hist	 (1085, 756)
--- 1.1210179328918457 seconds Histofram ---


## Classification ##

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train_hist, 
                                                    labels, 
                                                    test_size=0.3)

print('X_train.shape\t', X_train.shape)
print('X_test.shape\t', X_test.shape)
print('y_train.shape\t', y_train.shape)
print('y_test.shape\t', y_test.shape)


X_train.shape	 (759, 756)
X_test.shape	 (326, 756)
y_train.shape	 (759,)
y_test.shape	 (326,)


In [15]:


model_svm = SVC()
best_svm = model_svm.fit(X_train, y_train)
start_PR_time = time.time()## точка отсчета времени
Y_pred_svm = best_svm.predict(X_test)
print("Accuracy:",accuracy_score(y_test, Y_pred_svm))
print(classification_report(y_test, Y_pred_svm))

print("--- %s seconds Predict time ---" % (time.time() - start_PR_time)) ## вывод время работы программы

Accuracy: 0.7147239263803681
              precision    recall  f1-score   support

           0       0.87      0.61      0.72        64
           1       0.66      0.91      0.76        69
           2       0.91      0.66      0.76        61
           3       0.77      0.79      0.78        70
           4       0.51      0.58      0.55        62

    accuracy                           0.71       326
   macro avg       0.74      0.71      0.71       326
weighted avg       0.74      0.71      0.72       326

--- 0.06505918502807617 seconds Predict time ---


In [16]:
print("--- %s seconds ALL  ---" % (time.time() - startall_time)) ## вывод время работы программы

--- 83.39075660705566 seconds ALL  ---


In [17]:
start_KNN_time = time.time()## точка отсчета времени

model_knn = KNeighborsClassifier(n_neighbors=1)
model_knn.fit(X_train, y_train)

print('KNN train acc\t:', model_knn.score(X_train, y_train))
print('KNN test acc\t:', model_knn.score(X_test, y_test))

print("--- %s seconds KNN ---" % (time.time() - start_KNN_time)) ## вывод время работы программы

KNN train acc	: 1.0
KNN test acc	: 0.7822085889570553
--- 0.034030914306640625 seconds KNN ---


In [24]:
start_TEST_time = time.time()## точка отсчета времени

foil_test_dir = os.getcwd() + "\\Desert_test"
corduroy_test_dir = os.getcwd() + "\\DenseResidential_test"
cork_test_dir = os.getcwd() + "\\Forest_test"
catton_test_dir = os.getcwd() + "\\Meadow_test"
linen_test_dir = os.getcwd() + "\\River_test"


foil_test_list = LaP_images(foil_test_dir)
corduroy_test_list = LaP_images(corduroy_test_dir)
cork_test_list = LaP_images(cork_test_dir)
catton_test_list = LaP_images(catton_test_dir)
linen_test_list = LaP_images(linen_test_dir)


all_test_images = np.vstack((foil_test_list,
                        corduroy_test_list,
                        cork_test_list,
                        catton_test_list,
                        linen_test_list
                        ))

labels_test = []
for i in range(len(foil_test_list)):
    labels_test.append(0)
for i in range(len(corduroy_test_list)):
    labels_test.append(1)
for i in range(len(cork_test_list)):
    labels_test.append(2)
for i in range(len(catton_test_list)):
    labels_test.append(3)
for i in range(len(linen_test_list)):
    labels_test.append(4)

    
labels_test = np.array(labels_test)

In [25]:
X_test_lbp = extract_lbp(all_test_images)

In [26]:
X_test_lbp.shape

(165, 150, 150)

In [27]:
X_test_hist = create_histograms(X_test_lbp, sub_images_num=3, bins_per_sub_images=84)

In [28]:
start_TEPR_time = time.time()## точка отсчета времени

X_sc_test = X_test_hist

Ytest = labels_test
Y_pred_test = best_svm.predict(X_sc_test)

In [29]:
print(classification_report(Ytest, Y_pred_test))
print("--- %s seconds Test_predict ---" % (time.time() - start_TEPR_time))
print("--- %s seconds test ---" % (time.time() - start_TEST_time))

              precision    recall  f1-score   support

           0       0.95      0.55      0.69        33
           1       0.66      0.88      0.75        33
           2       0.77      0.73      0.75        33
           3       0.58      0.88      0.70        33
           4       0.71      0.45      0.56        33

    accuracy                           0.70       165
   macro avg       0.73      0.70      0.69       165
weighted avg       0.73      0.70      0.69       165

--- 0.05004525184631348 seconds Test_predict ---
--- 12.265142440795898 seconds test ---
